In [1]:
# Dependencies and Setup
import pandas as pd

In [2]:
# File to Load (Remember to Change These)
school_data_to_load = "resources/schools_complete.csv"
student_data_to_load = "resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)


In [3]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## District Summary

In [4]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["school_name"].unique())


In [5]:
#calculate the total number of students
student_count = school_data_complete["student_name"].count()


In [6]:
# Calculate the total budget (take into account duplicate data)
x = school_data_complete.groupby(["school_name"]).mean()["budget"]
total_budget = sum(x)

In [7]:
# Calculate the average (mean) math score
average_math_score = school_data_complete['math_score'].mean()


In [8]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete['reading_score'].mean()


In [9]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100


In [10]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) *100


In [11]:
# Use the following to calculate the percentage of students that passed math and reading

passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100


In [12]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary =  pd.DataFrame({
    "Total Unique Schools" : [school_count],
    "Total Students" : [student_count],
    "Total Budget" : [total_budget],
    "Average Math Score" : [average_math_score],
    "Average Reading Score" : [average_reading_score],
    "% passing math" : [passing_math_percentage],
    "% passing reading" : [passing_reading_percentage],
    "% passing overall" : [overall_passing_rate]
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Unique Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% passing math,% passing reading,% passing overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [13]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]
school_types = school_types.sort_index()


In [14]:
# Calculate the total student count
per_school_counts = school_data_complete["school_name"].value_counts()
per_school_counts = per_school_counts.sort_index()
per_school_counts = per_school_counts.rename("school_counts") #renamed for the later dataframe




In [15]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
 



In [16]:
per_school_capita = per_school_budget / per_school_counts
#series must be names in order to merge later
per_school_capita = per_school_capita.rename("per_capita")



In [17]:
# Calculate the average test scores
per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"] 


In [18]:
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]


In [19]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete.loc[(school_data_complete["math_score"] >= 70)]

In [20]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete.loc[(school_data_complete["reading_score"] >= 70)]

In [21]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]

In [22]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_math = per_school_passing_math.rename("math passing rate")
per_school_passing_reading = per_school_passing_reading.rename("reading passing rate")
overall_passing_rate = overall_passing_rate.rename("overall passing rate")

In [23]:
#merge in the above vatiables for creating a dataframe below
schools_merge = pd.merge(school_types, per_school_counts, left_index=True, right_index=True)
schools_merge = pd.merge(schools_merge, per_school_budget, left_index=True, right_index=True)
schools_merge = pd.merge(schools_merge, per_school_capita, left_index=True, right_index=True)
schools_merge = pd.merge(schools_merge, per_school_math, left_index=True, right_index=True)
schools_merge = pd.merge(schools_merge, per_school_reading, left_index=True, right_index=True)
schools_merge = pd.merge(schools_merge, per_school_passing_math, left_index=True, right_index=True)
schools_merge = pd.merge(schools_merge, per_school_passing_reading, left_index=True, right_index=True)
schools_merge = pd.merge(schools_merge, overall_passing_rate, left_index=True, right_index=True)


In [24]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.

per_school_summary =  pd.DataFrame(schools_merge)




In [25]:

# Formatting
per_school_summary["budget"] = per_school_summary["budget"].map("${:,.2f}".format)
per_school_summary["per_capita"] = per_school_summary["per_capita"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,type,school_counts,budget,per_capita,math_score,reading_score,math passing rate,reading passing rate,overall passing rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Highest-Performing Schools (by % Overall Passing)

In [26]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("overall passing rate", ascending=False)
top_schools.head()

,type,school_counts,budget,per_capita,math_score,reading_score,math passing rate,reading passing rate,overall passing rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [27]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values("overall passing rate")
bottom_schools.head()

,type,school_counts,budget,per_capita,math_score,reading_score,math passing rate,reading passing rate,overall passing rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [28]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby("school_name")["reading_score", "math_score"].mean()
tenth_graders_scores = tenth_graders.groupby("school_name")["reading_score", "math_score"].mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name")["reading_score", "math_score"].mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name")["reading_score", "math_score"].mean()

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

#Make sure each is a series or df
ninth_grade_math_scores = pd.DataFrame(ninth_grade_math_scores)
tenth_grader_math_scores = pd.DataFrame(tenth_grader_math_scores)
eleventh_grader_math_scores = pd.DataFrame(eleventh_grader_math_scores)
twelfth_grader_math_scores = pd.DataFrame(twelfth_grader_math_scores)



#Rename the series so that the titles are correct in the below dataframe
ninth_grade_math_scores = ninth_grade_math_scores.rename(columns = {"math_score": "ninth_grade_math_scores"})
tenth_grader_math_scores = tenth_grader_math_scores.rename(columns = {"math_score": "tenth_grade_math_scores"})
eleventh_grader_math_scores = eleventh_grader_math_scores.rename(columns = {"math_score": "eleventh_grade_math_scores"})
twelfth_grader_math_scores = twelfth_grader_math_scores.rename(columns = {"math_score": "twelfth_grade_math_scores"})

C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\119910869.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ninth_graders_scores = ninth_graders.groupby("school_name")["reading_score", "math_score"].mean()
C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\119910869.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tenth_graders_scores = tenth_graders.groupby("school_name")["reading_score", "math_score"].mean()
C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\119910869.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  eleventh_graders_scores = eleventh_graders.groupby("school_name")["reading_score", "math_score"].mean()
C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\119910869.py:11: FutureWarning: Indexing with multiple keys (implicitl

In [29]:

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.merge(ninth_grade_math_scores, tenth_grader_math_scores, right_index=True, left_index=True)
math_scores_by_grade = pd.merge(math_scores_by_grade, eleventh_grader_math_scores, right_index=True, left_index=True)
math_scores_by_grade = pd.merge(math_scores_by_grade, twelfth_grader_math_scores, right_index=True, left_index=True)


# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,ninth_grade_math_scores,tenth_grade_math_scores,eleventh_grade_math_scores,twelfth_grade_math_scores
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores by Grade

In [30]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.

# This was already done above, so we do not have to do it again here

ninth_graders_scores = ninth_graders.groupby("school_name")["reading_score", "math_score"].mean()
tenth_graders_scores = tenth_graders.groupby("school_name")["reading_score", "math_score"].mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name")["reading_score", "math_score"].mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name")["reading_score", "math_score"].mean()

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
#We have to rename the columns to match first
ninth_grade_reading_scores = ninth_grade_reading_scores.rename("9th")
tenth_grader_reading_scores = tenth_grader_reading_scores.rename("10th")
eleventh_grader_reading_scores = eleventh_grader_reading_scores.rename("11th")
twelfth_grader_reading_scores = twelfth_grader_reading_scores.rename("12th")

reading_scores_by_grade = pd.merge(ninth_grade_reading_scores, tenth_grader_reading_scores, left_index=True, right_index=True)
reading_scores_by_grade = pd.merge(reading_scores_by_grade, eleventh_grader_reading_scores, left_index=True, right_index=True)
reading_scores_by_grade = pd.merge(reading_scores_by_grade, twelfth_grader_reading_scores, left_index=True, right_index=True)


# # Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\340798745.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ninth_graders_scores = ninth_graders.groupby("school_name")["reading_score", "math_score"].mean()
C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\340798745.py:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tenth_graders_scores = tenth_graders.groupby("school_name")["reading_score", "math_score"].mean()
C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\340798745.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  eleventh_graders_scores = eleventh_graders.groupby("school_name")["reading_score", "math_score"].mean()
C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\340798745.py:14: FutureWarning: Indexing with multiple keys (implici

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [31]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [32]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [33]:
school_spending_df.head()

,type,school_counts,budget,per_capita,math_score,reading_score,math passing rate,reading passing rate,overall passing rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


In [34]:
#we need to remove the formatting and make it a float64 again
school_spending_df["budget"] = school_spending_df["budget"].str.replace("$", "").str.replace(",", "").astype(float)
school_spending_df["per_capita"] = school_spending_df["per_capita"].str.replace("$", "").str.replace(",", "").astype(float)


C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\2842929197.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  school_spending_df["budget"] = school_spending_df["budget"].str.replace("$", "").str.replace(",", "").astype(float)
C:\Users\EmGre\AppData\Local\Temp\ipykernel_27364\2842929197.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  school_spending_df["per_capita"] = school_spending_df["per_capita"].str.replace("$", "").str.replace(",", "").astype(float)


In [35]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['per_capita'], spending_bins, labels=labels, include_lowest=True)


In [36]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["math_score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["reading_score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["math passing rate"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["reading passing rate"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["overall passing rate"]



In [37]:
# Assemble into DataFrame
spending_summary = pd.merge(spending_math_scores, spending_reading_scores, left_index=True, right_index=True)
spending_summary = pd.merge(spending_summary, spending_passing_math, left_index=True, right_index=True)
spending_summary = pd.merge(spending_summary, spending_passing_reading, left_index=True, right_index=True)
spending_summary = pd.merge(spending_summary, overall_passing_spending, left_index=True, right_index=True)


# Display results
spending_summary

,math_score,reading_score,math passing rate,reading passing rate,overall passing rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Score by School Size

In [38]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [39]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["school_counts"], size_bins, labels = labels, include_lowest=True)

In [40]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["math_score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["reading_score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["math passing rate"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["reading passing rate"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["overall passing rate"]

In [41]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.merge(size_math_scores, size_reading_scores, left_index=True, right_index=True)
size_summary = pd.merge(size_summary, size_passing_math, left_index=True, right_index=True)
size_summary = pd.merge(size_summary, size_passing_reading, left_index=True, right_index=True)
size_summary = pd.merge(size_summary, size_overall_passing, left_index=True, right_index=True)



# Display results
size_summary

,math_score,reading_score,math passing rate,reading passing rate,overall passing rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [42]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["type"]).mean()["math_score"]
type_reading_scores = per_school_summary.groupby(["type"]).mean()["reading_score"]
type_passing_math = per_school_summary.groupby(["type"]).mean()["math passing rate"]
type_passing_reading = per_school_summary.groupby(["type"]).mean()["reading passing rate"]
type_overall_passing = per_school_summary.groupby(["type"]).mean()["overall passing rate"]


In [43]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.merge(type_math_scores,type_reading_scores, left_index=True, right_index=True)
type_summary = pd.merge(type_summary,type_passing_math, left_index=True, right_index=True)
type_summary = pd.merge(type_summary,type_passing_reading, left_index=True, right_index=True)
type_summary = pd.merge(type_summary,type_overall_passing, left_index=True, right_index=True)

# Display results
type_summary

,math_score,reading_score,math passing rate,reading passing rate,overall passing rate
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
